In [41]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split




In [42]:
data = pd.read_csv('hamlet.txt', sep='\t', header=None)

In [43]:
data

,0
0,[The Tragedie of Hamlet by William Shakespeare...
1,Actus Primus. Scoena Prima.
2,Enter Barnardo and Francisco two Centinels.
3,Barnardo. Who's there?
4,Fran. Nay answer me: Stand & vnfold
...,...
3961,"Becomes the Field, but heere shewes much amis."
3962,"Go, bid the Souldiers shoote."
3963,"Exeunt. Marching: after the which, a Peale of ..."
3964,off.


In [44]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data[0])
total_words = len(tokenizer.word_index) + 1

In [45]:
input_sequences = []
for line in data[0]:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [46]:
# Get the max sequence length
max_sequence_length = max([len(x) for x in input_sequences])

In [47]:
# Apply padding to the sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

In [48]:
x, y = input_sequences[:,:-1],input_sequences[:,-1]

In [49]:
# Remove the duplicates values from y
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [50]:
# Start splitting the data to train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [51]:
## Apply the early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [52]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train the model
history = model.fit(X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/50
644/644 [==============================] - 14s 20ms/step - loss: 6.9142 - accuracy: 0.0341 - val_loss: 6.6919 - val_accuracy: 0.0336
Epoch 2/50
644/644 [==============================] - 12s 18ms/step - loss: 6.4802 - accuracy: 0.0380 - val_loss: 6.7487 - val_accuracy: 0.0396
Epoch 3/50
644/644 [==============================] - 12s 18ms/step - loss: 6.3332 - accuracy: 0.0463 - val_loss: 6.8031 - val_accuracy: 0.0466
Epoch 4/50
644/644 [==============================] - 12s 18ms/step - loss: 6.1878 - accuracy: 0.0503 - val_loss: 6.8056 - val_accuracy: 0.0484
Epoch 5/50
644/644 [==============================] - 12s 18ms/step - loss: 6.0435 - accuracy: 0.0558 - val_loss: 6.8364 - val_accuracy: 0.0591
Epoch 6/50
644/644 [==============================] - 12s 18ms/step - loss: 5.8851 - accuracy: 0.0651 - val_loss: 6.9178 - val_accuracy: 0.0639


In [54]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [57]:
input_text="Barn. Last night of all, when yond same "
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction:{next_word}")

Input text:Barn. Last night of all, when yond same 
Next Word Prediction:king
